1. Installing Dependencies


In [1]:
# %pip install tensorflow==2.6 opencv-python mediapipe==0.9.1.0 scikit-learn matplotlib

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import os
import mediapipe as mp

2. Capturing video and Drawing keypoints using MP Holistic

In [2]:
# Keypoints Portion
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

3. Extract Keypoint Values

In [3]:
#Extracting Keypoints into a numpy array

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4) 
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) 
# why is it important to flatten the array?
# it is because we want to have a single row of data for each image

# why is it important to concatenate? 
# it is because we want to have a single row of data for each image

4. Setup Folders for Collection

In [4]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_Maura') 

# Actions that we try to detect
actions = np.array(['ako',  'ikaw', 'maganda', 'magandang umaga', 'hi'])
# actions = np.array(['ako',  'ikaw', 'maganda', 'magandang umaga', 'hi', 'hindi', 'oo', 'salamat', 'bakit', 'kamusta'])

# Thirty videos worth of data
no_sequences = 4

# Videos are going to be 30 frames in length
sequence_length = 30

In [5]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

5. Collect Videos and Store them under videos/signed_videos

In [7]:
actions = np.array(['ako',  'ikaw', 'maganda', 'magandang umaga', 'hi', 'hindi', 'oo', 'salamat', 'bakit', 'kamusta'])
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join("videos", "signed_videos", action))
        except:
            pass

In [21]:
import cv2
import os
import time

cap = cv2.VideoCapture(0)

print("Starting video collection")

for action in actions:
    print("Collection of", action,  "is starting in 1 second.")
    cv2.waitKey(500)
    for sequence in range(no_sequences):
        frames = []
        for frame_num in range(sequence_length):
            if frame_num==0:
                print("Collecting data for", action, "sequence:", sequence)
                cv2.waitKey(500)
            ret, frame = cap.read()
            if ret:
                frames.append(frame)
        out = cv2.VideoWriter(f'videos/signed_videos_temp/{action}/video_{sequence}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (640, 480))
        for frame in frames:
            out.write(frame)
            cv2.imshow('frame', frame)
        cv2.waitKey(500)
        out.release()

cap.release()
cv2.destroyAllWindows()

Starting video collection
Collection of ako is starting in 1 second.
Collection of ikaw is starting in 1 second.


KeyboardInterrupt: 

In [17]:
cap.release()
cv2.destroyAllWindows()

5. Collect Keypoint Values for Training and Testing

In [14]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                # print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic 
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,100), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,100), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [15]:
cap.release()
cv2.destroyAllWindows()

6. Arrange the folders: If there are already existing dataset on MP_Data similar to MP_Data_2, then run this cell to transfer the videos from MP_Data_2 to MP_Data. This is to avoid overwriting the existing dataset.

In [ ]:
import os
import shutil

source_folder = "MP_Data_2"
destination_folder = "MP_Data"

for folder_name in os.listdir(source_folder):
    source_path = os.path.join(source_folder, folder_name)
    destination_path = os.path.join(destination_folder, folder_name)
    if os.path.isdir(source_path) and os.path.exists(destination_path):
        max_number = max([int(file_name.split(".")[0]) for file_name in os.listdir(destination_path) if file_name.endswith(".mp4")])
        for file_name in os.listdir(source_path):
            if file_name.endswith(".mp4"):
                max_number += 1
                new_file_name = f"{max_number}.mp4"
                source_file = os.path.join(source_path, file_name)
                destination_file = os.path.join(destination_path, new_file_name)
                shutil.move(source_file, destination_file)
                
# This code will compare the two folders MP_Data and MP_Data_2. If there is a similar name folder in both MP_Data and MP_Data_2, then it will move all .mp4 files from the MP_Data_2 folder to the folder from MP_Data. The name convention will be followed when the .mp4 files are moved. The videos will be numbered after 14.mp4 so the next video is 15.mp4 and the next video transferred is 16.mp4 and so on.